# < Sklearn SelfTrainingClassifier tutorial >

In [ ]:
! git clone https://github.com/KU-DIC/LG_semi_supervised_learning_day17.git

In [ ]:
''' 기본 모듈 및 시각화 모듈 '''
from IPython.display import display
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

''' 데이터 전처리 모듈 '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

''' 모델 학습용 모듈 '''
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier

''' 결과 평가용 모듈 '''
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

## 1. 분석 데이터: Pistachio Dataset (이진 분류 문제)

### Task abstract : 피스타치오의 16가지 속성을 기반으로 피스타치오 종 분류

### 설명변수(X) : 피스타치오 속성

- AREA
- PERIMETER
- MAJOR_AXIS
- MINOR_AXIS
- ECCENTRICITY
- EQDIASQ
- SOLIDITY
- CONVEX_AREA
- EXTENT
- ASPECT_RATIO 
- ROUNDNESS
- COMPACTNESS
- SHAPEFACTOR_1
- SHAPEFACTOR_2
- SHAPEFACTOR_3
- SHAPEFACTOR_4

### 반응변수(Y) : 피스타치오 종

- Species : Kirmizi_Pistachio, Siit_Pistachio  

### 1-1 데이터 전처리 및 탐색적 데이터 분석

### 데이터 불러오기

In [ ]:
data =  pd.read_csv('/content/LG_semi_supervised_learning_day17/data/Pistachio.csv')

### 데이터 확인

In [ ]:
print('Data shape: {}'.format(data.shape))
data.head()

### 클래스 비율 확인

In [ ]:
data['Class'].value_counts()

### 결측치 확인

In [ ]:
data.isnull().sum()

### 1-2 학습 데이터(Training Dataset), 테스트 데이터(Testing Dataset) 정의

### 학습 데이터, 테스트 데이터 분리

In [ ]:
df_train, df_test = train_test_split(data, test_size=0.25, random_state=0)
print('Size of train dataframe: ', df_train.shape[0])
print('Size of test dataframe: ', df_test.shape[0])

##### Train Data를 Labeled Data, Unlabeled Data로 Split 
- Random_Mask column을 추가
- Labeled Data => -1, Unlabeled Data => 1을 가지도록 설정
- frac으로 Labeled Data의 비율을 설정
- New_Target column을 추가
- Random_Mask => -1인 데이터들은 New_Target에 실제 Class value 할당
- Random_Mask => 1인 데이터들은 New_Target에 -1을 value로 할당

In [ ]:
df_train['Random_Mask'] = True
df_train.loc[df_train.sample(frac=0.01, random_state=0).index, 'Random_Mask'] = False

In [ ]:
df_train['New_Target'] = df_train.apply(lambda x: x['Class'] if x['Random_Mask']==False else -1, axis=1)
df_train['New_Target'].value_counts()

## 2. Model Training, Testing

- Labeled Data만 사용해 학습한 Supervised Learning Model과 Unlabeled Data도 사용하여 학습한 Semi-Supervised Learning Model의 성능을 비교

### 2-1. Baseline Model Training
- Labeled Data만 사용한 Supervised Learning Model 구축

##### Labeled Data 의  Train Data, Test Data 정의

In [ ]:
df_train_labeled = df_train[df_train['New_Target']!=-1]

X_baseline = df_train_labeled.drop(['Class', 'Random_Mask', 'New_Target'], axis=1)
y_baseline = df_train_labeled['New_Target'].values

X_test = df_test.drop(['Class'], axis=1)
y_test = df_test['Class'].values

##### Baseline Model Training & Testing

##### Sklearn SVC Parameters
- kernel : 어떤 kernel trick을 활용할지 지정 ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed') 
- C, gamma : decision boundary 조절 역할 -> Margin을 크게하여 일반화 오차를 줄이는 것 vs 학습 데이터를 잘 분류하도록 하는 것

In [ ]:
# 모델 정의
model = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=0)

# 모델 학습
clf = model.fit(X_baseline, y_baseline)

# 모델 평가
print('---------- SVC Baseline Model - Evaluation on Test Data ----------')
accuracy_score_B = clf.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_B)
print(classification_report(y_test, clf.predict(X_test)))

### 2-2. Self-Training Model Training
- Unlabeled data, Labeled Data를 모두 활용해 Semi-Supervised Learining Model 구축

##### Self-Training Train Data, Test Data 정의

In [ ]:
X_train = df_train.drop(['Class', 'Random_Mask', 'New_Target'], axis = 1)
y_train = df_train['New_Target'].values

##### Self-Training & Testing

##### Sklearn SelfTrainingClassifier Parameters

- base_estimator : fit, predict_probability를 구현하는 추정기, 학습에 사용하는 모델

- criterion : pseudo labeled 데이터 중 훈련 데이터에 추가할 데이터를 선택하는 기준 ('threshold' -> 예측 확률이 threshold 이상인 데이터들을 훈련 데이터에 추가, 'k-best' -> 예측 확률이 가장 높은 k개의 데이터를 훈련 데이터로 추가)

- threshold : criterion이 'threshold'일 경우 임계값 지정

- k-best : criterion이 'k-best'일 경우 k값 지정

- max-iter : pseudo labeling, training 최대 반복 횟수를 지정

In [ ]:
# base_estimator 모델 정의
model_svc = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=0)

# self training classifier 모델 정의
self_training_model = SelfTrainingClassifier(base_estimator=model_svc,
                                             threshold=0.75,
                                             criterion='threshold',
                                             max_iter=10,
                                             verbose=True)
# self training classifier 모델 학습
clf_ST = self_training_model.fit(X_train, y_train)

In [ ]:
# self training classifier 모델 결과
print('---------- Self Training Model - Summary ----------')
print('Base Estimator: ', clf_ST.base_estimator_)
print('Classes: ', clf_ST.classes_)
print('Transduction Labels: ', clf_ST.transduction_)
print('Number of Features: ', clf_ST.n_features_in_)
print('Number of Iterations: ', clf_ST.n_iter_)
print('Termination Condition: ', clf_ST.termination_condition_)

In [ ]:
# self training classifier 모델 평가
print('---------- Self Training Model - Evaluation on Test Data ----------')
accuracy_score_ST = clf_ST.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_ST)
print(classification_report(y_test, clf_ST.predict(X_test)))